In [ ]:
# 회귀
  # - 단순 회귀 : 하나의 특성을 사용
  # - 다중 회귀 : 여러 개의 특성을 사용

# 특성 공학 : 기존의 특성을 사용해 새로운 특성을 뽑아내는 작업


In [1]:
# 데이터 준비

import pandas as pd

df = pd.read_csv('https://bit.ly/perch_csv_data')
perch_full = df.to_numpy()                              # 농어 데이터 : 길이,높이,두께

#print("perch_full :\n", perch_full, "\n")
#print("perch_full.shape :", perch_full.shape, "\n")

import numpy as np

perch_weight = np.array([5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 1000.0, 1000.0])

# 훈련세트와 테스트세트로 나누기

from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    perch_full, perch_weight, random_state=42
)

# 사이킷런의 변환기

# 사이킷런에는 변환기(transfomer)라 부르는 특성을 만들거나 전처리하는 클래스가 있다.

from sklearn.preprocessing import PolynomialFeatures # PolynomialFeatures 클래스는 기본적으로 각 특성을 제곱한 항을 추가하고, 특성끼리 서로 곱한 항을 추가한다.

###########################################################################

# 2개의 특성(2, 3)으로 이루어진 샘플 하나를 적용해보자.
# 2와 3을 각기 제곱한 4와 9가 추가되었고, 2와 3을 곱한 6이 추가되었으며, 1이 추가되었다.
# 1이 추가된 이유는 다음과 같다.
# 무게 = a * 길이 + b * 높이 + c * 두께 + d * 1
# 선형 방정식의 절편을 항상 값이 1인 특성과 곱해지는 계수라고 볼 수 있다.
# 특성은 (길이, 높이, 두께, 1)이 된다.

poly = PolynomialFeatures(include_bias=True)
poly.fit([[2, 3]])
print("include_bias=Tr :", poly.transform([[2, 3]]))
poly = PolynomialFeatures(include_bias=False)
poly.fit([[2, 3]])
print("include_bias=Fa :", poly.transform([[2, 3]]))
....

###########################################################################

# 훈련세트의 입력데이터에 적용

poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input)                             # fit()으로 훈련을 해야 변환이 가능하다.
train_poly = poly.transform(train_input)          # transform()으로 실제로 데이터를 변환한다.
print("train_poly.shape :", train_poly.shape)     # (42, 9)가 나온다. 이로써 특성이 9개가 되었다.

  # x0:길이 , x1:높이 ,x2: 두께)
    # ['x0' 'x1' 'x2' 'x0^2' 'x0 x1' 'x0 x2' 'x1^2' 'x1 x2' 'x2^2']
    # [0번째특성, 1번째특성, 2번째특성, 0번째특성의 제곱, 0번째특성*1번째특성, 0번째특성*2번째특성, 1번째특성의 제곱, 1번째특성*2번째특성, 2번째특성의 제곱]

print("get_feature_names_out() :", poly.get_feature_names_out())

# 테스트세트의 입력데이터에 적용
test_poly = poly.transform(test_input)

# 다중 회귀 모델 훈련하기

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_poly, train_target) # 훈련
print("결정계수 (tra) : ", lr.score(train_poly, train_target))  # 결정계수 점수 확인 : 훈련세트
print("결정계수 (tes) : ", lr.score(test_poly, test_target))    # 결정계수 점수 확인 : 테스트세트

poly = PolynomialFeatures(degree=5, include_bias=False) # degree 매개변수를 사용해 필요한 고차항의 최대 차수를 지정한다. 5제곱까지 특성을 만듦으로써 특성이 55개가 된다.
poly.fit(train_input)

train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)
print("tra~_poly.shape :", train_poly.shape)
print("tes~_poly.shape :", test_poly.shape)

lr.fit(train_poly, train_target)
print("결정계수 (tra) : ", lr.score(train_poly, train_target))  # 훈련세트로 결정계수 점수 확인
print("결정계수 (tes) : ", lr.score(test_poly, test_target))    # 테스트세트로 결정계수 점수 확인

# 훈련세트의 결정계수가 테스트세트의 결정계수보다 훨씬 큼으로 과대적합 발생

# 규제
  # : 머신러닝 모델이 훈련세트를 너무 과도하게 학습하지 못하도록 훼방하는 것을 말한다.
  # : 모델이 훈련 세트에 과대적합되지 않도록 만든다.
  # : 선형 회귀 모델의 경우 특성에 곱해지는 계수(또는 기울기)의 크기를 작게 만드는 일이다.

# 훈련 세트로 학습한 변환기를 사용해 테스트 세트까지 변환하기
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_poly)
train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)

# 이제 표준점수로 변환한 train_scaled와 test_scaled가 준비되었음
# 훈련 세트에서 학습한 평균과 표준편차는 StandardScaler 클래스 객체의 mean_, scale_ 속성에 저장함.
# 특성마다 계산하므로 55개의 평균과 표준편차가 들어있음.


include_bias=Tr : [[1. 2. 3. 4. 6. 9.]]
include_bias=Fa : [[2. 3. 4. 6. 9.]]
train_poly.shape : (42, 9)
get_feature_names_out() : ['x0' 'x1' 'x2' 'x0^2' 'x0 x1' 'x0 x2' 'x1^2' 'x1 x2' 'x2^2']
결정계수 (tra) :  0.9903183436982125
결정계수 (tes) :  0.9714559911594111
tra~_poly.shape : (42, 55)
tes~_poly.shape : (14, 55)
결정계수 (tra) :  0.9999999999996433
결정계수 (tes) :  -144.40579436844948


# 비교
## 릿지 회귀 & 라쏘 회귀

선형 회귀 모델에 규제를 추가한 모델을 릿지(ridge)와 라쏘(lasso)라고 부른다.

- 릿지 회귀는 계수를 제곱한 값을 기준으로 규제에 적용합니다.
- 라쏘 회귀는 계수의 절댓값을 기준으로 규제를 적용합니다.

<br>
<hr>
<br>

In [2]:
####################################################################################################################

# 릿지 회귀

from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.fit(train_scaled, train_target)
print(ridge.score(train_scaled, train_target))    # 훈련세트로 결정계수 점수 확인
print(ridge.score(test_scaled, test_target))      # 테스트세트로 결정계수 점수 확인

# hyperparameter 하이퍼파라미터
  # - 머신러닝 모델이 학습할 수 없고 사람이 알려줘야 하는 파라미터이다.
  # - 적절한 하이퍼파라미터 값을 찾는 방법은 하이퍼파라미터 값에 대한 결정계수 값의 그래프를 그리는 것이다.

import matplotlib.pyplot as plt

train_score = []
test_score = []

# alpha(hyperparameter) 값을 0.001부터 100까지 10배씩 늘려가며 릿지 회귀 모델을 훈련하고, 훈련세트와 테스트세트의 점수를 파이썬 리스트에 저장한다.

alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]

for alpha in alpha_list:
  ridge = Ridge(alpha=alpha)                                    # 릿지 모델을 만듭니다.
  ridge.fit(train_scaled, train_target)                         # 릿지 모델을 훈련합니다.
  train_score.append(ridge.score(train_scaled, train_target))   # 훈련 점수를 저장합니다.
  test_score.append(ridge.score(test_scaled, test_target))      # 테스트 점수를 저장합니다.

  # alpha 값을 0.001부터 100까지 10배씩 늘렸기 때문에 이대로 그래프를 그래면 한쪽이 너무 촘촘해진다.
  # alpha_list에 있는 6개의 값을 동일한 간격으로 나타내기 위해 로그 함수로 바꾸어 지수로 표현한다.
  # [0.001, 0.01, 0.1, 1, 10, 100] -> [-3, -2, -1, 0, 1, 2]

  # 파란색 : 훈련 세트 그래프
  # 주황색 : 테스트 세트 그래프

plt.plot(np.log10(alpha_list), train_score)
plt.plot(np.log10(alpha_list), test_score)
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.show()

# alpha 값을 0.1로 해서 최종 모델을 훈련

ridge = Ridge(alpha=0.1)
ridge.fit(train_scaled, train_target)
print(ridge.score(train_scaled, train_target))
print(ridge.score(test_scaled, test_target))

####################################################################################################################

# 라쏘 회귀

from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(train_scaled, train_target)
print(ridge.score(train_scaled, train_target))    # 훈련세트로 결정계수 점수 확인
print(ridge.score(test_scaled, test_target))      # 테스트세트로 결정계수 점수 확인

# hyperparameter 하이퍼파라미터
  # - 머신러닝 모델이 학습할 수 없고 사람이 알려줘야 하는 파라미터이다.
  # - 적절한 하이퍼파라미터 값을 찾는 방법은 하이퍼파라미터 값에 대한 결정계수 값의 그래프를 그리는 것이다.

import matplotlib.pyplot as plt

train_score = []
test_score = []

# alpha(hyperparameter) 값을 0.001부터 100까지 10배씩 늘려가며 릿지 회귀 모델을 훈련하고, 훈련세트와 테스트세트의 점수를 파이썬 리스트에 저장한다.

alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]

for alpha in alpha_list:
  lasso = Lasso(alpha=alpha, max_iter=10000)                    # 라쏘 모델을 만듭니다.
  lasso.fit(train_scaled, train_target)                         # 라쏘 모델을 훈련합니다.
  train_score.append(lasso.score(train_scaled, train_target))   # 훈련 점수를 저장합니다.
  test_score.append(lasso.score(test_scaled, test_target))      # 테스트 점수를 저장합니다.

  # alpha 값을 0.001부터 100까지 10배씩 늘렸기 때문에 이대로 그래프를 그래면 한쪽이 너무 촘촘해진다.
  # alpha_list에 있는 6개의 값을 동일한 간격으로 나타내기 위해 로그 함수로 바꾸어 지수로 표현한다.
  # [0.001, 0.01, 0.1, 1, 10, 100] -> [-3, -2, -1, 0, 1, 2]

  # 파란색 : 훈련 세트 그래프
  # 주황색 : 테스트 세트 그래프

plt.plot(np.log10(alpha_list), train_score)
plt.plot(np.log10(alpha_list), test_score)
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.show()

# alpha 값을 0.1로 해서 최종 모델을 훈련

lasso = Lasso(alpha=10)
lasso.fit(train_scaled, train_target)
print(lasso.score(train_scaled, train_target))
print(lasso.score(test_scaled, test_target))

####################################################################################################################